# <center>CHALLENGE 4.1<center>

<p>Team Name:Group 1
<p>Student Names:Kaden Larussa, Brooks Schafer, Nick Solari
<p>Student W#s:w0762283, w0711688, w0755643

## Instructions
Use <b>generic</b> coding style unless hard-coded values are really necessary.<br>
Your code must be efficient and use self-explanatory naming.<br>
Use appropriate Python library methods for each task instead of using loops.<br>
Run your entire code and save <b>BEFORE</b> submitting. Then, submit this <b>saved</b> copy.

## Definitions

AGE_12   : Passengers with age in \[1, 13)<br>
AGE_TEEN : Passengers with age in \[13, 20)<br>
AGE_YOUNG: Passengers with age in \[20, 31)<br>
AGE_OTHER: Passengers with age >= 31

## Imports

In [63]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os

## Loading Data

Load "titanic.csv" into the data frame object "data"</br>
[train.csv](https://github.com/wesm/pydata-book/blob/2nd-edition/datasets/titanic/train.csv)

In [64]:
data = pd.read_csv('titanic.csv')

## Calculate Counts

Calculate joint counts for each {Survived_i, Age_j}, i = {Yes, No}, j = {_12, _Teen, _Young, _Other}:

In [65]:
# Define age categories as per the provided specifications
def age_category(age):
    if age <= 12:
        return '_12'
    elif age > 12 and age < 20:
        return '_Teen'
    elif age >= 20 and age < 40:
        return '_Young'
    else:
        return '_Other'

data['Age_Category'] = data['Age'].apply(age_category)

data['Survived'] = data['Survived'].map({1: 'Yes', 0: 'No'})

joint_counts = pd.crosstab(data['Survived'], data['Age_Category'])

joint_counts

Age_Category,_12,_Other,_Teen,_Young
Survived,,,,
No,29,227,56,237
Yes,40,113,39,150


Calculate marginal counts for survive categories:

In [66]:

marginal_counts_survived_corrected = data['Survived'].value_counts()

marginal_counts_survived_corrected


Survived
No     549
Yes    342
Name: count, dtype: int64

Calculate marginal counts for age categories:

In [67]:
marginal_counts_age = data['Age_Category'].value_counts()

marginal_counts_age


Age_Category
_Young    387
_Other    340
_Teen      95
_12        69
Name: count, dtype: int64

Calculate total count of samples:

In [68]:

total_count_samples = data.shape[0]  

total_count_samples


891

Store all of the counts calculated above into the data frame "Count"<br>
.    Column headers: Age categories and TOTAL<br>
.    Row headers: survive categories, TOTAL<br>
Print the object Count's content:

In [69]:


# Calculate the joint counts again to reset the previous manipulation
joint_counts = pd.crosstab(data['Survived'], data['Age_Category'])

# Add marginal totals for rows at the end of the joint_counts DataFrame
joint_counts.loc['TOTAL', :] = joint_counts.sum(axis=0)

# Add marginal totals for columns
joint_counts['TOTAL'] = joint_counts.sum(axis=1)

# The corrected "Count" DataFrame
Count = joint_counts


Count


Age_Category,_12,_Other,_Teen,_Young,TOTAL
Survived,,,,,
No,29.0,227.0,56.0,237.0,549.0
Yes,40.0,113.0,39.0,150.0,342.0
TOTAL,69.0,340.0,95.0,387.0,891.0


Save Counts into the text file "Count.csv" under the folder "OUTPUT".<br>
.    Column headers: Age categories and TOTAL<br>
.    Row headers: survive categories, TOTAL:

In [70]:
# Since there is no existing "OUTPUT" folder mentioned, it will be created in the specified directory if not present.
output_directory = 'OUTPUT/'
os.makedirs(output_directory, exist_ok=True)  # Create the OUTPUT directory if it doesn't exist

# Define the file path
output_file_path = os.path.join(output_directory, 'Count.csv')

# Save the Count dataframe to a CSV file
Count.to_csv(output_file_path, header=True, index=True)  # Include headers and index in the output file

output_file_path  # Return the path of the created file for access


'OUTPUT/Count.csv'

## Calculate Probabilities

Generate the Probability Table T2- P() as described. Store into the data frame object "P". Print the object's content:

In [71]:
# Transposing the DataFrame "P" back to its original form to match the provided image format
P = P.T

# Renaming the index to match the provided image for the probability table
P = P.rename(index={'Not Survived = 0': 'Not Survived', 'Survived = 1': 'Survived'})

# Renaming the columns to match the provided image for the probability table
P.columns = ['Age_12', 'Age_Other', 'Age_Teen', 'Age_Young', 'TOTAL']

# reorder because im retarded
P = P[['Age_12', 'Age_Teen', 'Age_Young', 'Age_Other', 'TOTAL']]

# Display the reordered 'P' DataFrame

P


,Age_12,Age_Teen,Age_Young,Age_Other,TOTAL
No,0.032548,0.265993,0.254770,0.062851,0.616162
Yes,0.044893,0.168350,0.126824,0.043771,0.383838
TOTAL,0.077441,0.434343,0.381594,0.106622,1.000000


Generate the Probability Table T3_1- P(Survive | Age) as described. Store into the data frame object "P_SgA". Print the object's content:

In [72]:
# Calculating P(Survive | Age) which is the probability of survival within each age category
# This requires dividing the count of survivors by the total count within each age category

# Isolating the counts of survivors and non-survivors for each age category from the 'Count' DataFrame
survivors_count = Count.loc['Yes', ['_12', '_Teen', '_Young', '_Other']]
total_age_count = Count.loc['TOTAL', ['_12', '_Teen', '_Young', '_Other']]

# Calculating the conditional probability of survival within each age category
P_SgA = survivors_count / total_age_count

# Creating a DataFrame from the series and transposing it to match the requested format
P_SgA = pd.DataFrame(P_SgA).T

# Renaming index and columns to match the structure provided in the image
P_SgA.index = ['Survived | Age']
P_SgA.columns = ['Age_12', 'Age_Teen', 'Age_Young', 'Age_Other']

# Adding a row for 'Not Survived | Age' by subtracting 'Survived | Age' from 1
P_SgA.loc['Not Survived | Age'] = 1 - P_SgA.loc['Survived | Age']

# Adding the 'TOTAL' row for marginal probabilities of survival, which we already have in DataFrame 'P'
P_SgA.loc['TOTAL'] = P.loc['TOTAL', ['Age_12', 'Age_Teen', 'Age_Young', 'Age_Other']]

# Displaying the "P_SgA" DataFrame
P_SgA


,Age_12,Age_Teen,Age_Young,Age_Other
Survived | Age,0.579710,0.410526,0.387597,0.332353
Not Survived | Age,0.420290,0.589474,0.612403,0.667647
TOTAL,0.077441,0.434343,0.381594,0.106622


Generate the Probability Table T3_2- P(Survive | Age) P(Age) as described. Store into the data frame object "P_SgA_A". Print the object's content:

Generate the Probability Table T4_1- P(Age | Survive) as described. Store into the data frame object "P_AgS". Print the object's content:

Generate the Probability Table T4_2- P(Age | Survive) P(Survive) as described. Store into the data frame object "P_AgS_S". Print the object's content:

## Compare

Compare the row TOTAL of T3_1- P(Survive | Age) with the row TOTAL of T2- P(). Explain:

Compare the row TOTAL of T3_2- P(Survive | Age) P(Age) with the row TOTAL of T2- P(). Explain:

Compare the column TOTAL of T4_1- P(Age | Survive) with the column TOTAL of T2- P(). Explain:

Compare the column TOTAL of T4_2- P(Age | Survive) P(Survive) with the column TOTAL of T2- P(). Explain:

## <center> REFERENCES </center>
List resources (book, internet page, etc.) that you used to complete this challenge.